# Abstract

Learn about the data.

# Environment

In [ ]:
import sys
from pathlib import Path

In [ ]:
sys.path.insert(0, str(Path('..')))

In [ ]:
from bokeh.io import output_notebook, push_notebook, show
from bokeh.models import LinearAxis, Range1d
from bokeh.plotting import figure
from ipywidgets import FloatSlider, IntSlider, interact
import numpy as np
from scipy import ndimage

In [ ]:
DATA_PATH = Path('.') / '..' / '..' / 'data'

# Library

In [ ]:
def moving_average(a, n=3):
    """Compute the moving average"""
    result = np.cumsum(a)
    result[n:] = result[n:] - result[:-n]
    return result[n-1:] / n

In [ ]:
def central_idxs_constraint(a, c=0.8, adj=0):
    """Return central indexes of regions meeting constraint
    
    Parameters
    ----------
    a: nd.array
        Convolution array
    c: float
        Constraint above which counts as a signal
    adj: int
        Adjustment to apply to the indicies, due to averaging
        
    Returns
    -------
    central_idxs: [int[,...]]
        List of central indexes of continous regions
        where the contraint has been met
    """
    central_idxs = []
    meets = np.abs(a) > c
    has_met = False
    for idx, met in enumerate(meets):
        if met:
            adj_idx = idx + adj
            if has_met:
                region.append(adj_idx)
            else:
                region = [adj_idx]
                has_met = True
        else:
            if has_met:
                central_idxs.append(int(np.mean(region)))
                has_met = False
    return central_idxs

In [ ]:
def calc_jump_means(data, jump_locations, data_window=3, pad=1):
    """Determine the means on each side of the jumps
    
    Parameters
    ----------
    data: np.array
        The data.
        
    jump_locations: list or np.array
        The jump locations.
        
    data_window: int
        Size of the averaging window
        
    pad: int
        Distance from the jump location to place the average
        windows on each side of the jump.
        
    Returns
    -------
    [((left_idx,  left_mean), (right_idx, right_mean))[,...]]
        Two pairs of index and mean, left and right of the jump.
    """
    jump_means = []
    half_window = data_window // 2
    for jump_loc in jump_locations:
        left_idx = jump_loc - half_window - pad
        left_mean = np.mean(data[jump_loc - pad - data_window:jump_loc - pad])
        right_idx = jump_loc + half_window + pad
        right_mean = np.mean(data[jump_loc + pad:jump_loc + pad + data_window])
        jump_means.append(((left_idx, left_mean), (right_idx, right_mean)))
    return jump_means


# Main

In [ ]:
output_notebook()

## Read in the data

In [ ]:
data = {}
for fname in DATA_PATH.glob('*.npy'):
    data[fname.stem] = np.load(fname)

In [ ]:
jump = data['ubot']
idx_none = 0
idx_jumps = 1
idx_many_missed = 80
idx_steep_but_positive = 86

In [ ]:
jump = data['utop']

# idx = 6 F'ed up data
# idx = 11 hmmm, questionable
# idx = 17 twang
# idx = 25 bad data

In [ ]:
# jump = data['uref']


In [ ]:
# jump = data['umid']

# idx = 4 just steep?

## Explore results

In [ ]:
jump_constraint = 0.8
data_window = 13
convolve_window = 5
jump_kernel = [-1., 1.]

In [ ]:
mva = moving_average(jump[0], n=data_window)
# mva_convolve = np.convolve(mva, jump_kernel, mode='valid')
mva_convolve = ndimage.sobel(mva)
mva_convolve_mva = moving_average(mva_convolve, n=convolve_window)
jump_locations = central_idxs_constraint(mva_convolve_mva, jump_constraint, 
                                         adj=data_window / 2 + convolve_window / 2 + 1)
jump_means = calc_jump_means(jump[0], jump_locations, data_window=data_window)

In [ ]:
p_edge = figure(plot_width=950, tooltips=[("x", "$x"), ("value", "@y")])
p_edge.extra_y_ranges = {'convolve': Range1d(start=-2.0, end=2.0)}
p_edge.add_layout(LinearAxis(y_range_name='convolve'), 'right')
render_locs = p_edge.vbar(jump_locations, width=2, top=np.amax(jump[0]), bottom=np.amin(jump[0]), 
                         line_color='red', fill_color='red')
convolve = p_edge.circle(range(len(mva_convolve_mva)), mva_convolve_mva, size=2, y_range_name='convolve',
                        fill_color='green', line_color=None)
render_edge = p_edge.circle(range(len(jump[0])), jump[0], size=3, fill_color=None)

# Graph the means
left, right = zip(*jump_means)
idxs, means = zip(*left, *right)
render_means = p_edge.dash(idxs, means, size=20, line_color='orange')

In [ ]:
def update_p_edge(idx, cutoff=1.5, data_window=data_window, convolve_window=convolve_window):
    # Calculate
    mva = moving_average(jump[idx], n=data_window)
    mva_convolve = ndimage.sobel(mva)
    mva_convolve_mva = moving_average(mva_convolve, n=convolve_window)
    jump_locations = central_idxs_constraint(mva_convolve_mva, cutoff, adj=data_window / 2 + convolve_window / 2 + 1)
    jump_means = calc_jump_means(jump[idx], jump_locations, data_window=data_window)

    # Plot data and convolution
    render_edge.data_source.data['y'] = jump[idx]
    convolve.data_source.data['x'] = range(len(mva_convolve_mva)) 
    convolve.data_source.data['y'] = mva_convolve_mva
    
    # Plot out jump locations
    render_locs.data_source.data['x'] = jump_locations
    render_locs.glyph.top = np.amax(jump[idx])
    render_locs.glyph.bottom = np.amin(jump[idx])
    
    # Plot out jump means
    left, right = zip(*jump_means)
    idxs, means = zip(*left, *right)
    render_means.data_source.data['x'] = idxs
    render_means.data_source.data['y'] = means
    
    push_notebook()

In [ ]:
show(p_edge, notebook_handle=True)

In [ ]:
interact(update_p_edge, idx=IntSlider(min=0, max=data['ubot'].shape[0] - 1, value=0), 
         cutoff=FloatSlider(min=0., max=5., value=jump_constraint), 
         data_window=IntSlider(min=1, max=100, value=data_window), 
         convolve_window=IntSlider(min=1, max=100, value=convolve_window))

# Test & Utilities

## Plot a line

In [ ]:
p_nav = figure(plot_width=950, tooltips=[("x", "$x"), ("value", "@y")])

In [ ]:
render = p_nav.circle(range(40000), data['ubot'][0], size=4)

In [ ]:
def update_p_nav(idx):
    render.data_source.data['y'] = data['ubot'][idx]
    push_notebook()

In [ ]:
show(p_nav, notebook_handle=True)

In [ ]:
interact(update_p_nav, idx=(0, data['ubot'].shape[0] - 1))

## Moving Average

In [ ]:
mva_none = moving_average(jump[idx_none], n=10)

In [ ]:
p_mva_none = figure(plot_width=950)
p_mva_none.circle(range(len(mva_none)), mva_none, size=2)
show(p_mva_none)

In [ ]:
mva_jumps = moving_average(jump[idx_jumps], n=10)

In [ ]:
p_mva_jumps = figure(plot_width=950)
p_mva_jumps.circle(range(len(mva_jumps)), mva_jumps, size=2)
show(p_mva_jumps)

## Jump detection with basic convolution

In [ ]:
kernel = [-1., 1.]
# kernel = [1., -4., 1.]
# kernel = [4., -20., 4.]
# kernel = [-1., -2., 16, -2., -1.]

In [ ]:
c_none = np.convolve(jump[idx_none], kernel, mode='valid')

In [ ]:
p_convolve = figure(plot_width=950)

In [ ]:
p_convolve.circle(range(len(c_none)), c_none, size=2)

In [ ]:
show(p_convolve)

In [ ]:
c_jumps = np.convolve(mva_jumps, kernel, mode='valid')

In [ ]:
p_jumps = figure(plot_width=950)

In [ ]:
p_jumps.circle(range(len(c_jumps)), c_jumps, size=2)

In [ ]:
show(p_jumps)

In [ ]:
central_idxs = central_idxs_constraint(c_jumps, 0.7, adj=5+1)

In [ ]:
central_idxs

## Try the sobel

In [ ]:
s = ndimage.sobel(mva_jumps)

In [ ]:
p_sobel = figure()

In [ ]:
p_sobel.circle(range(len(s)), s, size=2)

In [ ]:
show(p_sobel)

In [ ]:
s_avg = moving_average(s, n=3)

In [ ]:
p_sobel_avg = figure()
p_sobel_avg.circle(range(len(s_avg)), s_avg, size=2)
show(p_sobel_avg)